In [1]:
# Librerias a utilizar
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib


In [2]:
path = '/content/bankloan.csv'
data = pd.read_csv(path)
data.head()

,ID,Age,Experience,Income,ZIP.Code,Family,CCAvg,Education,Mortgage,Personal.Loan,Securities.Account,CD.Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [3]:
data['Experience'].describe()

count    5000.000000
mean       20.104600
std        11.467954
min        -3.000000
25%        10.000000
50%        20.000000
75%        30.000000
max        43.000000
Name: Experience, dtype: float64

Observamos que el año mínimo de experiencia laboral es -3, lo cuál parece ser una inconsistencia de datos y no será tomado en cuenta

In [4]:
valid_experience = data[(data['Experience'] >= -2) & (data['Experience'] <= 43)]['Experience']
valid_experience

0        1
1       19
2       15
3        9
4        8
        ..
4995     3
4996     4
4997    39
4998    40
4999     4
Name: Experience, Length: 4996, dtype: int64

In [5]:
# Definir los límites de los grupos
bins = [0, 5, 15, 25, 35, 43]

# Asignar a cada cliente a su grupo correspondiente
data['Experience_Group'] = pd.cut(valid_experience, bins=bins, labels=['Sin Experiencia', 'Grupo 1', 'Grupo 2', 'Grupo 3', 'Grupo 4'])

# Imprimir los datos con la columna de grupos de experiencia laboral
print(data[['ID', 'Experience', 'Experience_Group']])


        ID  Experience Experience_Group
0        1           1  Sin Experiencia
1        2          19          Grupo 2
2        3          15          Grupo 1
3        4           9          Grupo 1
4        5           8          Grupo 1
...    ...         ...              ...
4995  4996           3  Sin Experiencia
4996  4997           4  Sin Experiencia
4997  4998          39          Grupo 4
4998  4999          40          Grupo 4
4999  5000           4  Sin Experiencia

[5000 rows x 3 columns]


In [7]:
modelo = joblib.load('RandomForestClassifier.pkl')
# Cargar el modelo desde el archivo .pkl
modelo = joblib.load('RandomForestClassifier.pkl')

# Iterar sobre cada grupo de datos (cada grupo de experiencia laboral)
for grupo, datos_grupo in data.groupby('Experience_Group'):
    print("Grupo de Experiencia Laboral:", grupo)

    # Dividir los datos del grupo en características y variable objetivo
    X_grupo = datos_grupo[['Income', 'CCAvg', 'Education', 'Mortgage', 'CD.Account']]
    y_grupo = datos_grupo['Personal.Loan']

    # Dividir los datos del grupo en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X_grupo, y_grupo, test_size=0.2, random_state=42)

    # Hacer predicciones en el conjunto de prueba
    predicciones = modelo.predict(X_test)

    # Calcular métricas de evaluación
    reporte = classification_report(y_test, predicciones)
    print("Reporte de clasificación:")
    print(reporte)

    # Generar matriz de confusión
    matriz_confusion = confusion_matrix(y_test, predicciones)
    print("Matriz de confusión:")
    print(matriz_confusion)
    print("\n")


Grupo de Experiencia Laboral: Sin Experiencia
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        95
           1       1.00      0.93      0.97        15

    accuracy                           0.99       110
   macro avg       0.99      0.97      0.98       110
weighted avg       0.99      0.99      0.99       110

Matriz de confusión:
[[95  0]
 [ 1 14]]


Grupo de Experiencia Laboral: Grupo 1
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       220
           1       0.90      0.90      0.90        21

    accuracy                           0.98       241
   macro avg       0.95      0.95      0.95       241
weighted avg       0.98      0.98      0.98       241

Matriz de confusión:
[[218   2]
 [  2  19]]


Grupo de Experiencia Laboral: Grupo 2
Reporte de clasificación:
              precision    recall  f1-score   support

   

### Interpretación de los Resultados por Grupo de Experiencia Laboral:

#### Sin Experiencia:
- La precisión para clientes que no aceptan préstamos personales es del 99%, y para aquellos que sí aceptan es del 100%.
- El recall para clientes que no aceptan préstamos personales es del 100%, y para aquellos que sí aceptan es del 93%.
- La puntuación F1 es alta tanto para clientes que aceptan como para aquellos que no, lo que indica un buen equilibrio entre precisión y recall.
- La matriz de confusión muestra 95 predicciones correctas de clientes que no aceptan préstamos personales y 14 predicciones correctas de clientes que sí aceptan. Solo hubo un falso positivo y ningún falso negativo.

#### Grupo 1:
- Las métricas de precisión, recall y F1-score son altas tanto para clientes que no aceptan como para aquellos que sí aceptan préstamos personales.
- La matriz de confusión muestra que la mayoría de las predicciones son correctas, con solo dos falsos positivos y dos falsos negativos.

#### Grupo 2:
- Similar al grupo anterior, las métricas de precisión, recall y F1-score son altas para ambos tipos de clientes.
- La matriz de confusión muestra una predicción casi perfecta, con solo dos falsos negativos.

#### Grupo 3:
- Las métricas de precisión, recall y F1-score son altas, indicando un buen rendimiento del modelo.
- La matriz de confusión muestra solo dos falsos positivos y dos falsos negativos.

#### Grupo 4:
- Aunque la precisión para clientes que no aceptan préstamos personales sigue siendo alta, la precisión para aquellos que sí aceptan es un poco menor.
- El recall para clientes que sí aceptan es alto, pero menor que en otros grupos.
- La matriz de confusión muestra algunos falsos positivos y falsos negativos, lo que indica que el modelo podría estar clasificando erróneamente algunos casos.

